# Simulazione Protocollo ECDH (Elliptic Curve Diffie-Hellman)

In questo notebook simuleremo una comunicazione sicura tra due dispositivi IoT: un **Sensore (Alice)** e un **Gateway (Bob)**.

Utilizzeremo il protocollo **ECDH** per generare una chiave segreta condivisa attraverso un canale non sicuro (come internet o onde radio), senza mai trasmettere la chiave stessa.

### Obiettivi:
1. Generare chiavi Pubbliche e Private su curve ellittiche.
2. Simulare lo scambio delle chiavi pubbliche.
3. Calcolare il "Segreto Condiviso" in modo indipendente.
4. Derivare una chiave simmetrica (AES) per cifrare i dati.

In [ ]:
# Installiamo la libreria crittografica standard per Python
!pip install cryptography

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet
import base64

print("Librerie importate correttamente.")

## Passo 1: Generazione delle Coppie di Chiavi

Ogni dispositivo deve generare la propria coppia di chiavi:
* **Chiave Privata:** Deve rimanere SEGRETA sul dispositivo. Non viene mai trasmessa.
* **Chiave Pubblica:** Viene derivata dalla privata e può essere inviata a chiunque.

Useremo la curva ellittica **SECP256R1** (nota anche come NIST P-256), standard molto comune nell'IoT.

In [ ]:
# --- DISPOSITIVO A (Sensore) ---
# Genera la chiave privata
private_key_A = ec.generate_private_key(ec.SECP256R1())
# Estrae la chiave pubblica da inviare
public_key_A = private_key_A.public_key()

# --- DISPOSITIVO B (Gateway) ---
# Genera la chiave privata
private_key_B = ec.generate_private_key(ec.SECP256R1())
# Estrae la chiave pubblica da inviare
public_key_B = private_key_B.public_key()

print("✅ Chiavi generate per entrambi i dispositivi.")

### Visualizziamo le chiavi pubbliche
Queste sono le informazioni che viaggiano "in chiaro" sulla rete. Anche se un hacker le intercetta, non può risalire alle chiavi private.

In [ ]:
def print_public_key(name, pub_key):
    pem = pub_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    print(f"--- Chiave Pubblica di {name} ---")
    print(pem.decode('utf-8'))

print_public_key("Device A", public_key_A)
print_public_key("Device B", public_key_B)

## Passo 2: Lo Scambio (Handshake)

Ora avviene la magia di Diffie-Hellman. 
1. Il Device A prende la chiave pubblica di B (ricevuta dalla rete) e la combina con la propria chiave privata.
2. Il Device B prende la chiave pubblica di A (ricevuta dalla rete) e la combina con la propria chiave privata.

Matematicamente: `PrivA * PubB` == `PrivB * PubA`.

In [ ]:
# Il Device A calcola il segreto condiviso
shared_secret_A = private_key_A.exchange(ec.ECDH(), public_key_B)

# Il Device B calcola il segreto condiviso
shared_secret_B = private_key_B.exchange(ec.ECDH(), public_key_A)

print(f"Lunghezza del segreto calcolato da A: {len(shared_secret_A)} bytes")
print(f"Lunghezza del segreto calcolato da B: {len(shared_secret_B)} bytes")

# Verifica fondamentale
assert shared_secret_A == shared_secret_B
print("\n✅ SUCCESSO: I due segreti sono IDENTICI!")
print(f"Valore esadecimale (primi 20 char): {shared_secret_A.hex()[:20]}...")

## Passo 3: Derivazione della Chiave (KDF)

Il "segreto condiviso" ottenuto sopra è un punto matematico sulla curva. Non è ancora una buona chiave crittografica (potrebbe avere pattern statistici deboli).

Dobbiamo passarlo attraverso una **KDF (Key Derivation Function)** per ottenere una chiave simmetrica pulita, ad esempio per l'algoritmo AES.

In [ ]:
def derive_key(shared_secret):
    # Usiamo HKDF standard con SHA256
    return HKDF(
        algorithm=hashes.SHA256(),
        length=32, # 32 bytes = 256 bits (per AES-256)
        salt=None, # In protocolli reali, il salt è scambiato durante l'handshake
        info=b'iot-handshake', # Info contesto applicativo
    ).derive(shared_secret)

aes_key_A = derive_key(shared_secret_A)
aes_key_B = derive_key(shared_secret_B)

# Codifichiamo in base64 per usarla con la libreria Fernet (che simula AES)
fernet_key_A = base64.urlsafe_b64encode(aes_key_A)
fernet_key_B = base64.urlsafe_b64encode(aes_key_B)

print(f"Chiave AES finale (Device A): {fernet_key_A}")
print(f"Chiave AES finale (Device B): {fernet_key_B}")

## Passo 4: Comunicazione Cifrata

Ora che entrambi hanno la stessa chiave simmetrica, possono scambiarsi dati cifrati. 
Il **Device A** invia la temperatura. Il **Device B** la riceve e la decifra.

In [ ]:
# Inizializziamo il modulo di cifratura simmetrica
cipher_suite_A = Fernet(fernet_key_A)
cipher_suite_B = Fernet(fernet_key_B)

# --- INVIO (Device A) ---
messaggio_chiaro = b"Temperatura: 24.5 C"
messaggio_cifrato = cipher_suite_A.encrypt(messaggio_chiaro)

print(f"📡 Device A invia: {messaggio_cifrato}")

# ... Il messaggio viaggia sulla rete ...

# --- RICEZIONE (Device B) ---
try:
    messaggio_decifrato = cipher_suite_B.decrypt(messaggio_cifrato)
    print(f"🔓 Device B decifra: {messaggio_decifrato.decode('utf-8')}")
except Exception as e:
    print("❌ Errore nella decifratura!")

### Conclusione

Abbiamo dimostrato come due dispositivi possano creare una chiave segreta comune partendo solo dalle rispettive chiavi pubbliche, senza mai scambiarsi la chiave privata o la chiave segreta finale.

Questo è il fondamento di **DTLS, TLS, HTTPS e SSH**.